In [4]:
import ase
from ase.io import read, write, lammpsdata
from ase.build import make_supercell
import numpy as np

In [5]:
#atoms = lammpsdata.read_lammps_data("/Users/iananderson/Documents/Documents/gradschool/TaC_Screw_Dislocations/modified.lmp")
atoms = read("/Users/iananderson/Downloads/CONTCAR", format='vasp')

In [6]:
transformation = np.array([
    [2,   0.0, 0.0],
    [12, 5, 4],
    [0.0,  0.0, 2.0]
])

# Apply the transformation
new_atoms = make_supercell(atoms, transformation)

# Save to file for LAMMPS
write("/Users/iananderson/Documents/Documents/gradschool/TaC_Screw_Dislocations/vasp.out", atoms, format='lammps-data')

In [25]:
def calculateTiltFactor(B, C, gamma, beta):
    b_mag = 4.455*np.linalg.norm(B)
    print(b_mag)
    c_mag = 4.455*np.linalg.norm(C)
    bx = b_mag*np.cos(np.deg2rad(gamma))
    by = b_mag*np.sin(np.deg2rad(gamma))
    cx = c_mag*np.cos(np.deg2rad(beta))
    cy = (np.dot(B, C)-bx*cx)/by
    cz = np.sqrt(c_mag**2-cx**2-cy**2)
    return bx, by, cx, cy, cz




In [7]:
def edge_dislocation_strain(x, y, x1, y1, x2, y2, burgers, nu):
    """
    Calculates the strain tensor components from an edge dislocation dipole in 2D.
    
    Parameters
    ----------
    x, y : float or ndarray
        Coordinates at which to evaluate the strain field.
    x1, y1 : float
        Position of the positive dislocation.
    x2, y2 : float
        Position of the negative dislocation.
    burgers : float
        Magnitude of Burgers vector (assumed in x-direction).
    nu : float
        Poisson's ratio.
    
    Returns
    -------
    exx, eyy, exy : float or ndarray
        Strain tensor components at each (x, y).
    """
    def single_dislocation_strain(x, y, x0, y0, b, nu):
        X = x - x0
        Y = y - y0
        r2 = X**2 + Y**2
        r4 = r2**2
        pre = b / (4 * np.pi * (1 - nu))
        
        # strain components from classical elasticity
        exx = -pre * ( (Y * (3 * X**2 + Y**2)) / r4 )
        eyy =  pre * ( (Y * (X**2 - Y**2)) / r4 )
        exy =  pre * ( (X * (X**2 - Y**2)) / r4 )
        
        return exx, eyy, exy

    # strain from positive dislocation
    exx1, eyy1, exy1 = single_dislocation_strain(x, y, x1, y1, burgers, nu)
    # strain from negative dislocation (same magnitude, opposite sign)
    exx2, eyy2, exy2 = single_dislocation_strain(x, y, x2, y2, -burgers, nu)

    # superpose the two fields
    exx = exx1 + exx2
    eyy = eyy1 + eyy2
    exy = exy1 + exy2

    return exx, eyy, exy

In [8]:
edge_dislocation_strain(15.8,7.74,-5.51,0,5.52,0,3.14,0.24)

(0.021286430409980545, -0.0004513333531326346, 0.004811104506074328)